# Task3 Subtask 2: Is it a Bargain

In this task, we wanna combine our implementations of task 1 and task 2 to build a bargain indicator. The mechanism is as follows:



In [211]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import euclidean_distances

In [212]:
import preprocess as ourprep
import joblib

In [213]:
df_sample = pd.read_csv('../task-2-recommendation/data/sg-property-prices-simplified.csv')
df_sample.head(5)

,listing_id,title,property_name,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,price
0,799762,hdb flat for sale in 524 ang mo kio avenue 5,hdb-ang mo kio,hdb 3 rooms,1980.0,2.0,2.0,732,ang mo kio,419000.0
1,896907,4 bed condo for sale in kopar at newton,kopar at newton,condo,2023.0,4.0,4.0,1528,novena,3727500.0
2,445021,4 bed condo for sale in nouvel 18,nouvel 18,condo,2014.0,4.0,3.0,2476,newton,8013600.0
3,252293,hdb flat for sale in 467 jurong west street 41,hong kah ville,hdb,1985.0,3.0,2.0,1302,jurong west,682500.0
4,926453,hdb flat for sale in 664b punggol drive,waterway sunbeam,Hdb 5 Rooms,2016.0,3.0,2.0,1184,punggol,764400.0


In [214]:
data = df_sample.copy()
data["property_type"] = data["property_type"].apply(ourprep.preprocess_property_type)

In [215]:
data = data.drop(['listing_id', 'property_name', 'title', 
                  ], axis=1)

In [216]:
# fill na by mean
data[['built_year', 'num_beds', 'num_baths']] = data[['built_year', 'num_beds', 'num_baths']].fillna(
    data[['built_year', 'num_beds', 'num_baths']].mean())



In [217]:
# filter out outliers
data = data[(data['size_sqft'] <= data['size_sqft'].quantile(0.99)) & (data['size_sqft'] > 10)]
data.drop(data[data.price <= 0].index, inplace=True)

In [218]:
data = data.drop(['price', 
                  ], axis=1)

In [219]:
data.drop(data[data.isnull().T.any()].index, inplace=True)

In [220]:
print('After cleaning, there are now {} records.'.format(data.shape[0]))
data.isnull().sum()

After cleaning, there are now 1980 records.


property_type    0
built_year       0
num_beds         0
num_baths        0
size_sqft        0
planning_area    0
dtype: int64

In [221]:
data['built_year'] = data['built_year'].astype(int)
data['num_baths'] = data['num_baths'].astype(int)
data['num_beds'] = data['num_beds'].astype(int)

In [222]:
labelEnc1 = preprocessing.LabelEncoder()
labelEnc2 = preprocessing.LabelEncoder()
labelEnc1.fit(data.planning_area)
labelEnc2.fit(data.property_type)
data.planning_area = labelEnc1.transform(data.planning_area)
data.property_type = labelEnc2.transform(data.property_type)

In [223]:
X = data.to_numpy()
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

In [224]:
from sklearn.metrics.pairwise import cosine_similarity

In [225]:
# This is basically the function we implemented in task 2. The only difference is we drop the price to get most similar houses (except the price)

def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters

    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = None
    
    # Extract all **kwargs input parameters
    # and set the used paramaters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value

    vec = row
    df = pd.DataFrame([row])
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!It is different from task 2 that we drop the column of price
    df = df.drop(['price', 
                  ], axis=1)
    df["property_type"] = df["property_type"].apply(ourprep.preprocess_property_type)
    df = df.drop(['listing_id', 'property_name', 'title',
                  ], axis=1)
    df['built_year'] = df['built_year'].astype(int)
    df['num_baths'] = df['num_baths'].astype(int)
    df['num_beds'] = df['num_beds'].astype(int)
    df.planning_area = labelEnc1.transform(df.planning_area)
    df.property_type = labelEnc2.transform(df.property_type)
    index = data[data == df.iloc[0]].dropna().index.tolist()[0]
    sim_matrix = np.zeros((X.shape[0], 1))
    for i in range(X.shape[0]):
        cos_sim = cosine_similarity(X[index].reshape(1, -1), X[i].reshape(1, -1))[0][0]
        sim_matrix[i] = cos_sim
    _k_sort = np.argpartition(sim_matrix.flatten(), -k)[-k:]
    _k_sort = _k_sort[np.argsort(sim_matrix[_k_sort].flatten())][::-1]

    # Here, we just return the input row k times
    # Ideally, you recommendations will be much better
    index_list = data.iloc[_k_sort].index.tolist()
    df_result = pd.DataFrame([row] * k, index=None)
    for i in range(k):
        df_result.iloc[i] = df_sample.iloc[index_list[i]]
    # Return the dataset with the k recommendations
    df_result.reset_index(drop=True, inplace=True)
    return df_result


In [226]:
# Pick a row id of choice
#row_id = 37
row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df_sample.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
df = pd.DataFrame([row])
df

,listing_id,title,property_name,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,price
20,942099,hdb flat for sale in 275a bishan street 24,natura loft,hdb 4 rooms,2011.0,3.0,2.0,1022,bishan,945000.0


In [227]:
k = 10

df_recommendations = get_top_recommendations(row, k=k)

df_recommendations.head(k)

,listing_id,title,property_name,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,price
0,942099,hdb flat for sale in 275a bishan street 24,natura loft,hdb 4 rooms,2011.0,3.0,2.0,1022,bishan,945000.0
1,706957,hdb flat for sale in 54 havelock road,havelock view,hdb,2013.0,3.0,2.0,958,bukit merah,1050000.0
2,458413,hdb flat for sale in 275a bishan street 24,natura loft,hdb 3 rooms,2011.0,3.0,2.0,1292,bishan,1344000.0
3,709639,hdb flat for sale in 53 havelock road,havelock view,Hdb 5 Rooms,2013.0,3.0,2.0,1227,bukit merah,1365000.0
4,513288,hdb flat for sale in 53 havelock road,havelock view,hdb 5 rooms,2013.0,3.0,2.0,1227,bukit merah,1365000.0
5,837248,hdb flat for sale in 633c senja road,senja green,hdb,2013.0,3.0,2.0,1001,bukit panjang,659400.0
6,908051,hdb flat for sale in 807c chai chee road,ping yi greens,hdb 3 rooms,2016.0,3.0,2.0,1001,bedok,829500.0
7,213501,hdb flat for sale in 635a senja road,senja gateway,hdb,2015.0,3.0,2.0,990,bukit panjang,606900.0
8,862524,hdb flat for sale in 185 bedok north road,vista 8,hdb,2005.0,3.0,2.0,990,bedok,661500.0
9,447266,hdb flat for sale in 442b fajar road,fajar hills,Hdb,2016.0,3.0,2.0,996,bukit panjang,572200.0


In [228]:
# original training data
train = pd.read_csv('../data/train.csv')

<mark> We can find that df only has few features. In order to put the df into our regression model of task 1, we need to fill the column of df according to original training set by using groupby and merge to get the latitude, longtitude and other important features.

In [229]:
def func(x):
    if len(x.mode()) == 0:
        return np.NaN
    else:
        return x.mode()

In [230]:
#group training data by property_name and impute below features with mean or mode
group = train.groupby('property_name').agg({'lat':'mean', 'lng':'mean', 'available_unit_types':lambda x: func(x)
                                            , 'furnishing':lambda x: func(x), 'subzone':lambda x: func(x), 
                                            'tenure':lambda x: func(x)})
group

,lat,lng,available_unit_types,furnishing,subzone,tenure
property_name,,,,,,
1 canberra,1.437693,103.830245,"studio, 1, 2, 3, 4 br",unspecified,yishun west,99-year leasehold
1 lorong lew lian,1.350893,103.875325,NaN,unspecified,upper paya lebar,NaN
1 pine close,1.308046,103.883179,NaN,unspecified,aljunied,NaN
10 evelyn,1.316290,103.840576,"studio, 1, 2, 3 br",unspecified,moulmein,freehold
10 pine close,1.309068,103.882052,NaN,unspecified,aljunied,NaN
...,...,...,...,...,...,...
yunnan gardens,1.338284,103.691089,"studio, 4, 5, 6, 9 br",unspecified,yunnan,freehold
zedge,1.322402,103.848192,"1, 2, 3 br",unspecified,balestier,freehold
zenith,1.292621,103.830516,"studio, 1, 2, 3 br",partial,chatsworth,999-year leasehold


In [231]:
#merge features and df according to property_name
df = df.merge(group[['lat','lng', 'available_unit_types', 'furnishing', 'subzone', 'tenure']], how='left', on=['property_name'])
df

,listing_id,title,property_name,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,price,lat,lng,available_unit_types,furnishing,subzone,tenure
0,942099,hdb flat for sale in 275a bishan street 24,natura loft,hdb 4 rooms,2011.0,3.0,2.0,1022,bishan,945000.0,1.356235,103.843449,"3, 4 br",unspecified,marymount,99-year leasehold


In [232]:
df["property_type"] = df["property_type"].apply(ourprep.preprocess_property_type)
df = df.drop(['listing_id', 'property_name', 'title', 'price'
                  ], axis=1)
# fill na by mean
df[['built_year', 'num_beds', 'num_baths']] = df[['built_year', 'num_beds', 'num_baths']].fillna(
    train[['built_year', 'num_beds', 'num_baths']].mean())
# fill na by mode
df['available_unit_types'] = df['available_unit_types'].fillna(train['available_unit_types'].mode()[0])

In [233]:
groups = train.groupby(train['property_type'])
mode_by_group = groups['tenure'].transform(lambda x: x.mode()[0])
df['tenure'] = df['tenure'].fillna(mode_by_group)

# preprocess tenure
df["tenure"] = df["tenure"].apply(ourprep.preprocess_tenure)

top10Dist = euclidean_distances(df[['lat', 'lng']], df[['lat', 'lng']]).argsort(axis=1)[:, :200]

for index, row in df[df['planning_area'].isna()].iterrows():
    paMode = df.loc[top10Dist[index]][~df['planning_area'].isna()]['planning_area'].mode()[0]
    szMode = df.loc[top10Dist[index]][~df['subzone'].isna()]['subzone'].mode()[0]
    df.at[index, 'planning_area'] = paMode
    df.at[index, 'subzone'] = szMode

    
df.astype({'subzone':str, 'planning_area':str})

,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,lat,lng,available_unit_types,furnishing,subzone,tenure
0,hdb,2011.0,3.0,2.0,1022,bishan,1.356235,103.843449,"3, 4 br",unspecified,marymount,tenure-100


In [234]:
df["property_type"] = df["property_type"].str.lower()
df['built_year'] = df['built_year'].astype(int)
df['num_baths'] = df['num_baths'].astype(int)
df['num_beds'] = df['num_beds'].astype(int)
df = df.astype({'tenure': 'string'})
df = df.astype({'furnishing': 'string'})
df = df.astype({'subzone': 'string'})

In [235]:
commercials = pd.read_csv('../data/auxiliary-data/sg-commerical-centres.csv')
mrts = pd.read_csv('../data/auxiliary-data/sg-mrt-stations.csv')
primary_schools = pd.read_csv('../data/auxiliary-data/sg-primary-schools.csv')
second_schools = pd.read_csv('../data/auxiliary-data/sg-secondary-schools.csv')
shoppings = pd.read_csv('../data/auxiliary-data/sg-shopping-malls.csv')
subzones = pd.read_csv('../data/auxiliary-data/sg-subzones.csv')

In [236]:
normalizer = preprocessing.MinMaxScaler()
def normLatLng(X):
    X.lat = normalizer.fit_transform(X.lat.values.reshape(-1, 1))
    X.lng = normalizer.fit_transform(X.lng.values.reshape(-1, 1))
normLatLng(commercials)
normLatLng(mrts)
normLatLng(primary_schools)
normLatLng(second_schools)
normLatLng(shoppings)

In [237]:
df['shop_dist'] = euclidean_distances(df[['lat', 'lng']], shoppings[['lat', 'lng']]).min(axis=1)
df['mrt_dist'] = euclidean_distances(df[['lat', 'lng']], mrts[['lat', 'lng']]).min(axis=1)
df['commercials_dist'] = euclidean_distances(df[['lat', 'lng']], commercials[['lat', 'lng']]).min(axis=1)
df['second_school_dist'] = euclidean_distances(df[['lat', 'lng']], second_schools[['lat', 'lng']]).min(axis=1)
df['primary_school_dist'] = euclidean_distances(df[['lat', 'lng']], primary_schools[['lat', 'lng']]).min(axis=1)

# subzone: compute population density within the subzone
subzones = subzones[subzones.population > 0]
subzones['density'] = subzones.population / subzones.area_size
subzones.rename(columns={'name': 'subzone'}, inplace=True)

In [238]:
df = df.merge(subzones[['subzone', 'density']], how='left', on='subzone')

In [239]:
df[['density']] = df[['density']].fillna(
    subzones[['density']].mean())

In [240]:
df_available_unit_types = df['available_unit_types'].apply(ourprep.parse_available_unit_types)
df['available_unit_type_is_studio'] = df_available_unit_types.apply(lambda x: x['type'])
df['available_unit_type_min_room'] = df_available_unit_types.apply(lambda x: x['min'])
df['available_unit_type_max_room'] = df_available_unit_types.apply(lambda x: x['max'])
df = df.drop(['available_unit_types'], axis=1)

In [241]:
df = df.drop(['lat', 'lng'], axis=1)
df

,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,furnishing,subzone,tenure,shop_dist,mrt_dist,commercials_dist,second_school_dist,primary_school_dist,density,available_unit_type_is_studio,available_unit_type_min_room,available_unit_type_max_room
0,hdb,2011,3,2,1022,bishan,unspecified,marymount,tenure-100,102.846836,102.847001,102.846863,102.847414,102.846733,15080.698539,other,3,4


In [242]:
#load the best regression model from task1
rgs = joblib.load('../prediction/rgs.pkl')

In [243]:
df

,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,furnishing,subzone,tenure,shop_dist,mrt_dist,commercials_dist,second_school_dist,primary_school_dist,density,available_unit_type_is_studio,available_unit_type_min_room,available_unit_type_max_room
0,hdb,2011,3,2,1022,bishan,unspecified,marymount,tenure-100,102.846836,102.847001,102.846863,102.847414,102.846733,15080.698539,other,3,4


In [244]:
row_data = df.copy()
row_data

,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,furnishing,subzone,tenure,shop_dist,mrt_dist,commercials_dist,second_school_dist,primary_school_dist,density,available_unit_type_is_studio,available_unit_type_min_room,available_unit_type_max_room
0,hdb,2011,3,2,1022,bishan,unspecified,marymount,tenure-100,102.846836,102.847001,102.846863,102.847414,102.846733,15080.698539,other,3,4


In [245]:
clean = pd.read_csv('../clean/train.csv')

In [246]:
labelEnc = preprocessing.LabelEncoder()
clean.planning_area = labelEnc.fit_transform (clean.planning_area)
row_data.planning_area = labelEnc.transform(row_data.planning_area)
labelEnc = preprocessing.LabelEncoder()
clean.subzone = labelEnc.fit_transform(clean.subzone)
row_data.subzone = labelEnc.transform(row_data.subzone)
labelEnc = preprocessing.LabelEncoder()
clean.furnishing = labelEnc.fit_transform(clean.furnishing)
row_data.furnishing = labelEnc.transform(row_data.furnishing)
labelEnc = preprocessing.LabelEncoder()
clean.tenure = labelEnc.fit_transform(clean.tenure)
row_data.tenure = labelEnc.transform(row_data.tenure)
labelEnc = preprocessing.LabelEncoder()
clean.property_type = labelEnc.fit_transform(clean.property_type)
row_data.property_type = labelEnc.transform(row_data.property_type)
labelEnc = preprocessing.LabelEncoder()
clean.available_unit_type_is_studio = labelEnc.fit_transform(clean.available_unit_type_is_studio)
row_data.available_unit_type_is_studio = labelEnc.transform(row_data.available_unit_type_is_studio)

In [247]:
clean.drop(axis=1,columns=['price'],inplace=True)

In [248]:
scaler = preprocessing.StandardScaler().fit(clean)
row_data = scaler.transform(row_data)

In [249]:
row_data

array([[ 6.04076163e-01,  3.95862241e+03, -1.29489792e+02,
        -8.83973180e-01,  7.27604240e+02, -1.03694350e+00,
         5.42063995e-01,  2.90805190e-02, -1.47351738e+00,
        -5.57933972e-02, -5.57743369e-02, -5.58341408e-02,
        -5.57850915e-02, -5.57890406e-02,  6.21139025e-03,
        -8.64924049e-01,  1.53136825e+00, -7.19126781e-01]])

In [250]:
#according to the model of task1, we can predict a reasonable price.
reasonable_price = rgs.predict(row_data)
reasonable_price

array([590911.])

In [251]:
#by comparing the predicted price with the recommendation results, it can give some intuitive advice to the guests
df_recommendations.head(k)

,listing_id,title,property_name,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,price
0,942099,hdb flat for sale in 275a bishan street 24,natura loft,hdb 4 rooms,2011.0,3.0,2.0,1022,bishan,945000.0
1,706957,hdb flat for sale in 54 havelock road,havelock view,hdb,2013.0,3.0,2.0,958,bukit merah,1050000.0
2,458413,hdb flat for sale in 275a bishan street 24,natura loft,hdb 3 rooms,2011.0,3.0,2.0,1292,bishan,1344000.0
3,709639,hdb flat for sale in 53 havelock road,havelock view,Hdb 5 Rooms,2013.0,3.0,2.0,1227,bukit merah,1365000.0
4,513288,hdb flat for sale in 53 havelock road,havelock view,hdb 5 rooms,2013.0,3.0,2.0,1227,bukit merah,1365000.0
5,837248,hdb flat for sale in 633c senja road,senja green,hdb,2013.0,3.0,2.0,1001,bukit panjang,659400.0
6,908051,hdb flat for sale in 807c chai chee road,ping yi greens,hdb 3 rooms,2016.0,3.0,2.0,1001,bedok,829500.0
7,213501,hdb flat for sale in 635a senja road,senja gateway,hdb,2015.0,3.0,2.0,990,bukit panjang,606900.0
8,862524,hdb flat for sale in 185 bedok north road,vista 8,hdb,2005.0,3.0,2.0,990,bedok,661500.0
9,447266,hdb flat for sale in 442b fajar road,fajar hills,Hdb,2016.0,3.0,2.0,996,bukit panjang,572200.0


In [252]:
#define the house is a bargain if tis price <=1.1*reasonable price
def is_bargain(data,r_price):
    data['bargain']=''
    for i in range(len(data)):
        if data.iloc[i,-2]<=1.1*r_price:
            data.iloc[i,-1]=1
        else:
            data.iloc[i,-1]=0
    return data

In [253]:
housing=df_recommendations.copy()

In [254]:
is_bargain(housing,reasonable_price).head(k)

,listing_id,title,property_name,property_type,built_year,num_beds,num_baths,size_sqft,planning_area,price,bargain
0,942099,hdb flat for sale in 275a bishan street 24,natura loft,hdb 4 rooms,2011.0,3.0,2.0,1022,bishan,945000.0,0
1,706957,hdb flat for sale in 54 havelock road,havelock view,hdb,2013.0,3.0,2.0,958,bukit merah,1050000.0,0
2,458413,hdb flat for sale in 275a bishan street 24,natura loft,hdb 3 rooms,2011.0,3.0,2.0,1292,bishan,1344000.0,0
3,709639,hdb flat for sale in 53 havelock road,havelock view,Hdb 5 Rooms,2013.0,3.0,2.0,1227,bukit merah,1365000.0,0
4,513288,hdb flat for sale in 53 havelock road,havelock view,hdb 5 rooms,2013.0,3.0,2.0,1227,bukit merah,1365000.0,0
5,837248,hdb flat for sale in 633c senja road,senja green,hdb,2013.0,3.0,2.0,1001,bukit panjang,659400.0,0
6,908051,hdb flat for sale in 807c chai chee road,ping yi greens,hdb 3 rooms,2016.0,3.0,2.0,1001,bedok,829500.0,0
7,213501,hdb flat for sale in 635a senja road,senja gateway,hdb,2015.0,3.0,2.0,990,bukit panjang,606900.0,1
8,862524,hdb flat for sale in 185 bedok north road,vista 8,hdb,2005.0,3.0,2.0,990,bedok,661500.0,0
9,447266,hdb flat for sale in 442b fajar road,fajar hills,Hdb,2016.0,3.0,2.0,996,bukit panjang,572200.0,1
